In [ ]:
import numpy as np

# Load the data
data = np.load("ddg-gd_drill_poisson_000.npy", allow_pickle=True)

# Iterate over grasps
for grasp in data:
    scale = grasp["scale"]
    final_pose = grasp["qpos"]
    initial_pose = grasp.get("qpos_st", None)  # May be missing after validation
    
    # print(f"Object scale: {scale}")
    # print(f"Final position: ({final_pose['WRJTx']}, {final_pose['WRJTy']}, {final_pose['WRJTz']})")
    # print(f"Final rotation (XYZ Euler): ({final_pose['WRJRx']}, {final_pose['WRJRy']}, {final_pose['WRJRz']})")
    # print(f"articulation: ({final_pose['Joint_index_DPflexion']})")

In [2]:
data[24]

{'scale': 0.15000000596046448,
 'qpos': {'F0_J0': 0.13476839661598206,
  'F0_J1': -0.7094214558601379,
  'F0_J2': 0.3894420862197876,
  'F0_J3': -0.007676619105041027,
  'F1_J0': 0.25747883319854736,
  'F1_J1': -1.2636785507202148,
  'F1_J2': 0.25788289308547974,
  'F1_J3': 0.12437296658754349,
  'F2_J0': -0.3061750531196594,
  'F2_J1': -0.90302973985672,
  'F2_J2': 0.05785776302218437,
  'F2_J3': -0.36681947112083435,
  'WRJRx': -0.5607176038748882,
  'WRJRy': 0.6759332513749957,
  'WRJRz': 0.4610713348919301,
  'WRJTx': -0.08972529321908951,
  'WRJTy': -0.2453954666852951,
  'WRJTz': -0.2297295778989792},
 'qpos_st': {'F0_J0': -0.1069839596748352,
  'F0_J1': -1.3393594026565552,
  'F0_J2': 0.06263192743062973,
  'F0_J3': -0.13199496269226074,
  'F1_J0': 0.33064407110214233,
  'F1_J1': -1.273848533630371,
  'F1_J2': 0.1412784904241562,
  'F1_J3': -0.21922919154167175,
  'F2_J0': -0.5894867181777954,
  'F2_J1': -0.7570757865905762,
  'F2_J2': 0.012896392494440079,
  'F2_J3': -0.2847253

In [7]:
import math

def euler_to_quaternion(roll, pitch, yaw):
    cy = math.cos(yaw * 0.5)
    sy = math.sin(yaw * 0.5)
    cp = math.cos(pitch * 0.5)
    sp = math.sin(pitch * 0.5)
    cr = math.cos(roll * 0.5)
    sr = math.sin(roll * 0.5)
    
    w = cr * cp * cy + sr * sp * sy
    x = sr * cp * cy - cr * sp * sy
    y = cr * sp * cy + sr * cp * sy
    z = cr * cp * sy - sr * sp * cy
    
    return (x, y, z, w)

def quaternion_inverse(q):
    x, y, z, w = q
    return (-x, -y, -z, w)

def quaternion_multiply(q1, q2):
    x1, y1, z1, w1 = q1
    x2, y2, z2, w2 = q2
    
    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2
    
    return (x, y, z, w)

# Your data
roll = -0.5028317411951178
pitch = -0.7766099381920684  
yaw = -0.9370075638944898

ee_robot = (0, 0, 0, 1)  # (x, y, z, w)

# Test different interpretations:
print("TEST DIFFERENT INTERPRETATIONS:")

# Option A: Direct conversion, assume same coordinate system
ee_object_A = euler_to_quaternion(roll, pitch, yaw)
object_robot_A = quaternion_multiply(ee_robot, quaternion_inverse(ee_object_A))
print(f"Option A (direct): w={object_robot_A[3]:.4f}, x={object_robot_A[0]:.4f}, y={object_robot_A[1]:.4f}, z={object_robot_A[2]:.4f}")

# Option B: Maybe the data is actually object orientation in ee frame?
# If ee_object means "object orientation as seen from ee frame"
object_robot_B = quaternion_multiply(ee_robot, ee_object_A)
print(f"Option B (object in ee frame): w={object_robot_B[3]:.4f}, x={object_robot_B[0]:.4f}, y={object_robot_B[1]:.4f}, z={object_robot_B[2]:.4f}")

# Option C: Maybe we need to swap coordinate axes
ee_object_C = euler_to_quaternion(pitch, yaw, roll)  # Try different order
object_robot_C = quaternion_multiply(ee_robot, quaternion_inverse(ee_object_C))
print(f"Option C (swapped): w={object_robot_C[3]:.4f}, x={object_robot_C[0]:.4f}, y={object_robot_C[1]:.4f}, z={object_robot_C[2]:.4f}")

TEST DIFFERENT INTERPRETATIONS:
Option A (direct): w=0.7573, x=0.3710, y=0.2232, z=0.4888
Option B (object in ee frame): w=0.7573, x=-0.3710, y=-0.2232, z=-0.4888
Option C (swapped): w=0.7573, x=0.4312, y=0.3208, z=0.3710
